In [21]:
import psycopg2
import os
from dotenv import load_dotenv
import uuid
import pandas as pd

# Load environment variables
load_dotenv()

# Database connection parameters
dbname = os.getenv("DB_NAME")
user = os.getenv("DB_USER")
password = os.getenv("DB_PASSWORD")
host = os.getenv("DB_HOST")
port = os.getenv("DB_PORT")  # Ensure you have DB_PORT in your .env file

# Path to your CSV file
csv_file_path = '../data/countries.csv'

# Read CSV file into DataFrame
df_countries = pd.read_csv(csv_file_path)

# Establishing Database Connection
try:
    conn = psycopg2.connect(dbname=dbname, user=user, password=password, host=host, port=port)
    cursor = conn.cursor()

    # Iterate over DataFrame rows
    for index, row in df_countries.iterrows():
      # Ensure all values are strings and escape single quotes
      name = str(row['name']).replace("'", "''") if pd.notna(row['name']) else ''
      iso_alpha2 = str(row['alpha-2']).replace("'", "''") if pd.notna(row['alpha-2']) else ''
      iso_alpha3 = str(row['alpha-3']).replace("'", "''") if pd.notna(row['alpha-3']) else ''
      region = str(row['region']).replace("'", "''") if pd.notna(row['region']) else ''
      sub_region = str(row['sub-region']).replace("'", "''") if pd.notna(row['sub-region']) else ''

      # Convert country-code to string or use NULL for NaN
      country_code = str(row['country-code']) if pd.notna(row['country-code']) else 'NULL'

      # Generate UUID for each country
      country_id = uuid.uuid4()

      # Prepare the SQL INSERT statement
      sql_statement = f"""
      INSERT INTO countries (country_id, name, iso_alpha2, iso_alpha3, country_code, region, sub_region, created, updated)
      VALUES ('{country_id}', '{name}', '{iso_alpha2}', '{iso_alpha3}', {country_code}, '{region}', '{sub_region}', CURRENT_TIMESTAMP, CURRENT_TIMESTAMP);
      """

      # Execute the SQL statement
      cursor.execute(sql_statement)

    # Commit the transaction
    conn.commit()
    print("All records inserted successfully")

except Exception as e:
    print(f"Error: {e}")

finally:
    # Close the cursor and connection
    if conn:
        cursor.close()
        conn.close()


All records inserted successfully
